In [1]:
import pandas as pd
import numpy as np 
np.set_printoptions(suppress=True)

import numpy as np
import pandas as pd
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# from diffprivlib.models import StandardScaler as DPStandardScaler
from flask_ngrok import run_with_ngrok
from flask import Flask
from flask import request, jsonify, send_file, make_response, flash, redirect
from werkzeug.utils import secure_filename
from flask_cors import CORS
import base64
import os
from pathlib import Path
from flask import Response
from time import sleep
from tqdm.notebook import tqdm
import glob
import json

In [ ]:
# DF_TARGET_Kag = pd.read_csv('kaggle-insurance.csv')
# DF_TARGET_Kag.head() 

# X = DF_TARGET_Kag.loc[:, 'age': 'region']
# X.sex = pd.Categorical(X.sex)
# X.smoker = pd.Categorical(X.smoker)
# X.region = pd.Categorical(X.region)
# X['sex'] = X.sex.cat.codes
# X['smoker'] = X.smoker.cat.codes
# X['region'] = X.region.cat.codes

# # X = DF_TARGET_Kag.loc[:, ['age','smoker', 'children']]
# # X.smoker = pd.Categorical(X.smoker)
# # X['smoker'] = X.smoker.cat.codes


# y = DF_TARGET_Kag.charges/10000

# display(X.head())
# display(y.head())

In [2]:
class RegressionDataset(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
class MultipleRegression(nn.Module):
    def __init__(self, num_features):
        super(MultipleRegression, self).__init__()
        self.layer_1 = nn.Linear(num_features, 32)
        self.layer_2 = nn.Linear(32, 64)
        self.layer_3 = nn.Linear(64, 128)
        self.layer_4 = nn.Linear(128, 256)
        self.layer_5 = nn.Linear(256, 256)
        self.layer_6 = nn.Linear(256, 128)
        self.layer_7 = nn.Linear(128, 64)
        self.layer_8 = nn.Linear(64, 32)
        self.layer_9 = nn.Linear(32, 16)
        self.layer_out = nn.Linear(16, 1)
        self.relu = nn.ReLU()
        self.relu2 = nn.LeakyReLU()

    def forward(self, inputs):
      x = self.relu(self.layer_1(inputs))
      x = self.relu2(self.layer_2(x))
      x = self.relu(self.layer_3(x))
      x = self.relu2(self.layer_4(x))
      x = self.relu(self.layer_5(x))
      x = self.relu2(self.layer_6(x))
      x = self.relu(self.layer_7(x))
      x = self.relu2(self.layer_8(x))
      x = self.relu(self.layer_9(x))
      x = self.layer_out(x)
      return (x)
    def predict(self, test_inputs):
      x = self.relu(self.layer_1(test_inputs))
      x = self.relu2(self.layer_2(x))
      x = self.relu(self.layer_3(x))
      x = self.relu2(self.layer_4(x))
      x = self.relu(self.layer_5(x))
      x = self.relu2(self.layer_6(x))
      x = self.relu(self.layer_7(x))
      x = self.relu2(self.layer_8(x))
      x = self.relu(self.layer_9(x))
      x = self.layer_out(x)
      return (x)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model_quick = MultipleRegression(6)
model_quick.to(device)
model_quick.load_state_dict(torch.load('model/model_quick.pth'))
model_quick.eval()


cuda:0


MultipleRegression(
  (layer_1): Linear(in_features=6, out_features=32, bias=True)
  (layer_2): Linear(in_features=32, out_features=64, bias=True)
  (layer_3): Linear(in_features=64, out_features=128, bias=True)
  (layer_4): Linear(in_features=128, out_features=256, bias=True)
  (layer_5): Linear(in_features=256, out_features=256, bias=True)
  (layer_6): Linear(in_features=256, out_features=128, bias=True)
  (layer_7): Linear(in_features=128, out_features=64, bias=True)
  (layer_8): Linear(in_features=64, out_features=32, bias=True)
  (layer_9): Linear(in_features=32, out_features=16, bias=True)
  (layer_out): Linear(in_features=16, out_features=1, bias=True)
  (relu): ReLU()
  (relu2): LeakyReLU(negative_slope=0.01)
)

In [7]:
# flask_ngrok_example.py
from flask import Flask
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route("/api/model/quick", methods = ['GET'])
def inference_quick():
    height = float(request.args.get('height'))
    weight = float(request.args.get('weight'))
    age = int(request.args.get('age'))
    children = int(request.args.get('children'))
    smoker = int(request.args.get('smoker'))
    region = int(request.args.get('region'))
    sex = int(request.args.get('sex'))
    BMI = weight / (height/100)**2


    map_sex = {
        'male': 0,
        'female': 1
    }
    map_smoker = {
        'no': 0,
        'yes': 1,
    }   
    map_region = {'northeast': 0, 'northwest': 1, 'southeast': 2, 'southwest': 3}

    X_test = np.array([[age, sex, BMI, children, smoker, region]])
    test_dataset = RegressionDataset(torch.from_numpy(X_test).float())
    test_loader = DataLoader(dataset=test_dataset, batch_size=1)
    ''''Testing'''
    # X_test = np.array([[45, 1, 15, 2, 1, 3]])
    test_dataset = RegressionDataset(torch.from_numpy(X_test).float())
    test_loader = DataLoader(dataset=test_dataset, batch_size=1)
    ''''Testing'''
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            x_test_pred = model_quick(X_batch)[0][0]
    result = {
        'premium_charge' : round(x_test_pred.item()*10000)
    }
    return json.dumps(result)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on https://hackaroo.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [15/Apr/2021 15:57:59] "GET /api/model/quick?height=150&weight=25&age=45&children=3&smoker=1&region=3&sex=1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 15:58:04] "GET /api/model/quick?height=150&weight=25&age=45&children=0&smoker=1&region=3&sex=1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 15:58:11] "GET /api/model/quick?height=150&weight=25&age=45&children=0&smoker=0&region=3&sex=1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 15:58:22] "GET /api/model/quick?height=150&weight=25&age=45&children=2&smoker=0&region=3&sex=1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/2021 15:58:34] "GET /api/model/quick?height=150&weight=55&age=25&children=2&smoker=0&region=3&sex=1 HTTP/1.1" 200 -
127.0.0.1 - - [15/Apr/20

In [40]:
X_test = np.array([[45, 1, 15, 2, 1, 3]])
test_dataset = RegressionDataset(torch.from_numpy(X_test).float())
test_loader = DataLoader(dataset=test_dataset, batch_size=1)
''''Testing'''
x_pred_list = []
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        x_test_pred = model_quick(X_batch)[0][0]

In [45]:
x_test_pred.item()

1.0353096723556519